In [ ]:
#
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from   udist_scatterplots_via_sectors          import UDistScatterPlotsViaSectors
from   udist_scatterplots_via_sectors_tile_opt import UDistScatterPlotsViaSectorsTileOpt
import time
import copy
import rtsvg
rt = rtsvg.RACETrack()
import random

In [ ]:
import os
_out_filename_ = 'udist_scatterplots_performance.parquet'
if os.path.exists(_out_filename_) == False:
    _pts_ = [(1_000, 2_000, 500),
            (  100,   200,  50),
            (8_000,   100, 400)]
    _its_ = [4, 16, 32]
    #_pts_ = [(1_000, 2_000, 500)]
    #_its_ = [4]
    perfs = {}
    for num_of_pts in _pts_:
        circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
        colors       = ['#ff0000','#006400','#0000ff']
        _xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
        for i in range(len(num_of_pts)):
            for j in range(num_of_pts[i]):
                a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
                x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
                _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])
        for _iterations_ in _its_:
            t0a = time.time()
            udspvs          = UDistScatterPlotsViaSectors       (_xvals_, _yvals_, iterations=_iterations_)
            t0b = time.time()
            k        = ('polars', num_of_pts, len(_xvals_), _iterations_, t0b-t0a)
            perfs[k] = copy.deepcopy(udspvs.time_lu)
            for _num_of_tiles_ in [16, 32, 64, 128, 256]:
                t1a = time.time()
                udspvs_tile_opt = UDistScatterPlotsViaSectorsTileOpt(_xvals_, _yvals_, iterations=_iterations_, num_of_tiles=_num_of_tiles_)
                t1b = time.time()
                k        = ('tile_opt', num_of_pts, len(_xvals_), _iterations_, t0b-t0a, _num_of_tiles_)
                perfs[k] = copy.deepcopy(udspvs_tile_opt.time_lu)
                print(f'{len(_xvals_):>5} Pts | {_iterations_:>3} Iters | {t0b-t0a:.2f}s Polars | {t1b-t1a:.2f}s Tile Opt ({_num_of_tiles_})')
    _lu_ = {'algorithm':[], 'pts_distro':[], 'num_of_pts':[], 'iters':[], 'time':[], 'num_of_tiles':[], 'subroutine':[], 'subroutine_time':[]}
    for _tuple_ in perfs:
        _algorithm_  = _tuple_[0]
        _pts_distro_ = _tuple_[1]
        _num_of_pts_ = _tuple_[2]
        _iters_      = _tuple_[3]
        _time_       = _tuple_[4]
        if   len(_tuple_) == 5: _num_of_tiles_ = None
        elif len(_tuple_) == 6: _num_of_tiles_ = _tuple_[5]
        else: raise Exception(f'unknown tuple {_tuple_}')
        for k in perfs[_tuple_]:
            _lu_['algorithm'].append(_algorithm_)
            _lu_['pts_distro'].append(_pts_distro_)
            _lu_['num_of_pts'].append(_num_of_pts_)
            _lu_['iters'].append(_iters_)
            _lu_['time'].append(_time_)
            _lu_['num_of_tiles'].append(_num_of_tiles_)
            _lu_['subroutine'].append(k)
            _lu_['subroutine_time'].append(perfs[_tuple_][k])
    _df_ = pl.DataFrame(_lu_)
    _df_.write_parquet(_out_filename_)
else:
    _df_ = pl.read_parquet(_out_filename_)

In [ ]:
# Results: Concat: 0.6618340015411377, Extend: 0.7158505916595459, Vstack: 0.7185516357421875
_lu0_, _lu1_ = {'x':[], 'y':[], 'z':[]}, {'x':[], 'y':[], 'z':[]}
for i in range(1_000_000):
    _lu0_['x'].append(random.random()), _lu0_['y'].append(random.random()), _lu0_['z'].append(random.random())
    _lu1_['x'].append(random.random()), _lu1_['y'].append(random.random()), _lu1_['z'].append(random.random())

t0 = time.time()
for i in range(10):
    df0, df1 = pl.DataFrame(_lu0_), pl.DataFrame(_lu1_)
    df_concat = pl.concat([df0, df1])
print(df_concat['x'].sum())
t1 = time.time()

t2 = time.time()
for i in range(10):
    df0, df1 = pl.DataFrame(_lu0_), pl.DataFrame(_lu1_)
    df_extend = df0.extend(df1)
print(df_extend['x'].sum())
t3 = time.time()

t4 = time.time()
for i in range(10):
    df0, df1 = pl.DataFrame(_lu0_), pl.DataFrame(_lu1_)
    df_vstack = df0.vstack(df1)
print(df_vstack['x'].sum())
t5 = time.time()
print(f"Concat: {t1-t0}, Extend: {t3-t2}, Vstack: {t5-t4}")

In [ ]:
_df_tile_opt_ = _df_.filter(pl.col('algorithm') == 'tile_opt')
rt.histogram(_df_tile_opt_, bin_by='subroutine', count_by='subroutine_time', w=512, h=384)

In [ ]:
#rt.smallMultiples(_df_tile_opt_, category_by='num_of_tiles', sm_type='histogram', sm_params={'bin_by':'subroutine', 'count_by':'subroutine_time'}, 
#                  y_axis_independent=False,
#                  w_sm_override=256, h_sm_override=320, w=1400)._repr_svg_()

# Original Performance (Contains Duplicate Sectors Error)

<svg id="smallMultiples_7250" x="0" y="0" width="1292" height="338" xmlns="http://www.w3.org/2000/svg"><rect width="1291" height="337" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><svg id="smallMultiples_7250_0" x="2" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_7250_0_encsvgid_s_sector:95:sums" width="242.0" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_hard:95:way:95:arctangents" width="203.190522275744" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_separate:95:easy:95:hard:95:way" width="166.60837303669862" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_join:95:sector:95:info" width="1.3653609156072568" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_ray:95:segment:95:intersections" width="0.7957615688380927" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_add:95:missing:95:sectors" width="0.7146604885021467" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_area:95:calc" width="0.3942849822462988" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_sector:95:uv:95:summation" width="0.39360732812658195" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_cross:95:join:95:tile:95:offsets" width="0.3315785809707383" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_join:95:sector:95:angles" width="0.20801036906810835" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_point:95:update" width="0.1546531744513556" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_tile:95:sums" width="0.14589368822292045" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_all:95:sectors" width="0.08379919248284232" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_prepare:95:sector:95:angles" width="0.07304990400883382" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_normalize" width="0.04510594897810522" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_xoyo:95:sector:95:creation" width="0.03785948691815691" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_determine:95:tile" width="0.02559152715799726" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_prepare:95:df" width="0.0032341849594581455" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_0_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">143.04</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">16</text><svg id="smallMultiples_7250_1" x="260" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_7250_1_encsvgid_s_sector:95:sums" width="194.75413467437554" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_hard:95:way:95:arctangents" width="163.85430077730774" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_separate:95:easy:95:hard:95:way" width="138.98559257937538" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_join:95:sector:95:info" width="3.8131988581048044" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_cross:95:join:95:tile:95:offsets" width="1.0543471203421877" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_ray:95:segment:95:intersections" width="0.8083264056411759" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_add:95:missing:95:sectors" width="0.7434716794598939" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_sector:95:uv:95:summation" width="0.42888688900324545" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_area:95:calc" width="0.39402360137155085" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_join:95:sector:95:angles" width="0.2035334148569552" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_point:95:update" width="0.15620290488466043" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_tile:95:sums" width="0.12613200332222532" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_all:95:sectors" width="0.0860536025275432" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_prepare:95:sector:95:angles" width="0.07237184652356952" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_normalize" width="0.04790490251186427" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_xoyo:95:sector:95:creation" width="0.035491327789074996" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_determine:95:tile" width="0.026432140958884114" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_prepare:95:df" width="0.0032902527705537658" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_1_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">143.04</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">32</text><svg id="smallMultiples_7250_2" x="518" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_7250_2_encsvgid_s_sector:95:sums" width="144.1272335146259" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_hard:95:way:95:arctangents" width="116.6588416239549" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_separate:95:easy:95:hard:95:way" width="106.63550505021833" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_join:95:sector:95:info" width="11.144245463256848" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_cross:95:join:95:tile:95:offsets" width="3.1319027508600263" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_ray:95:segment:95:intersections" width="0.7996233905893838" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_add:95:missing:95:sectors" width="0.7481575770246265" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_sector:95:uv:95:summation" width="0.4309799528289661" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_area:95:calc" width="0.39611263154179704" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_join:95:sector:95:angles" width="0.21365869682905775" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_point:95:update" width="0.1552057852513628" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_tile:95:sums" width="0.1426570830701776" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_all:95:sectors" width="0.0869809399211319" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_prepare:95:sector:95:angles" width="0.07140618940297302" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_normalize" width="0.04785569191507531" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_xoyo:95:sector:95:creation" width="0.04268010857573789" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_determine:95:tile" width="0.02918874111016086" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_prepare:95:df" width="0.0030655781606238348" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_2_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">143.04</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">64</text><svg id="smallMultiples_7250_3" x="776" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_7250_3_encsvgid_s_sector:95:sums" width="113.52967933862425" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_separate:95:easy:95:hard:95:way" width="82.95742666995196" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_hard:95:way:95:arctangents" width="76.08956351284068" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_join:95:sector:95:info" width="29.74565864411818" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_cross:95:join:95:tile:95:offsets" width="8.434823349775458" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_ray:95:segment:95:intersections" width="0.7897187495717366" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_add:95:missing:95:sectors" width="0.750923857949042" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_sector:95:uv:95:summation" width="0.42319983814974077" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_area:95:calc" width="0.38264869293344667" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_join:95:sector:95:angles" width="0.20847141588884427" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_tile:95:sums" width="0.15699471145430577" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_point:95:update" width="0.14803515591433516" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_all:95:sectors" width="0.08524485460490493" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_prepare:95:sector:95:angles" width="0.07268929520941307" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_xoyo:95:sector:95:creation" width="0.04623940616644114" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_normalize" width="0.04338519155268137" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_determine:95:tile" width="0.029532005191041242" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_prepare:95:df" width="0.0030369392067548487" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_3_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">143.04</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">128</text><svg id="smallMultiples_7250_4" x="1034" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_7250_4_encsvgid_s_sector:95:sums" width="111.05333514952282" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_separate:95:easy:95:hard:95:way" width="70.28005571951661" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_join:95:sector:95:info" width="65.69912839139043" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_hard:95:way:95:arctangents" width="46.559304087888805" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_cross:95:join:95:tile:95:offsets" width="16.93154878992486" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_ray:95:segment:95:intersections" width="0.7817647843415602" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_add:95:missing:95:sectors" width="0.7367391051073976" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_sector:95:uv:95:summation" width="0.38821231392943145" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_area:95:calc" width="0.3745337848498374" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_join:95:sector:95:angles" width="0.19522448794502226" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_tile:95:sums" width="0.16399471716476183" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_point:95:update" width="0.1514798976895625" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_all:95:sectors" width="0.08349989524663405" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_prepare:95:sector:95:angles" width="0.0725920841124775" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_normalize" width="0.0449252412128474" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_xoyo:95:sector:95:creation" width="0.033385356265835914" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_determine:95:tile" width="0.03209095622406724" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_prepare:95:df" width="0.003264034009969483" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_7250_4_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">143.04</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">256</text><rect width="1291" height="337" x="0" y="0" fill="none" fill-opacity="0.0" stroke="#000000" /></svg>

# Updated Performance (Fixed Duplicate Sectors)

<svg id="smallMultiples_45272" x="0" y="0" width="1292" height="338" xmlns="http://www.w3.org/2000/svg"><rect width="1291" height="337" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><svg id="smallMultiples_45272_0" x="2" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_45272_0_encsvgid_s_sector:95:sums" width="242.0" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_hard:95:way:95:arctangents" width="232.55577979067846" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_separate:95:easy:95:hard:95:way" width="178.67855574850518" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_join:95:sector:95:info" width="1.8434307097441387" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_ray:95:segment:95:intersections" width="1.7021740097402518" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_add:95:missing:95:sectors" width="1.566951378453381" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_sector:95:uv:95:summation" width="0.9534625800319408" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_area:95:calc" width="0.8179853503065201" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_cross:95:join:95:tile:95:offsets" width="0.5483218849582251" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_join:95:sector:95:angles" width="0.43093486011065524" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_tile:95:sums" width="0.3440224968471065" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_point:95:update" width="0.31656186930329394" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_all:95:sectors" width="0.17906756844686886" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_prepare:95:sector:95:angles" width="0.15017450322353737" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_xoyo:95:sector:95:creation" width="0.12877709034593532" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_normalize" width="0.09658024605805199" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_determine:95:tile" width="0.05408630936658885" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_prepare:95:df" width="0.006712997886482982" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_0_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">67.31</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">16</text><svg id="smallMultiples_45272_1" x="260" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_45272_1_encsvgid_s_sector:95:sums" width="215.32401912678475" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_hard:95:way:95:arctangents" width="196.07920947287556" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_separate:95:easy:95:hard:95:way" width="157.79869434861638" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_join:95:sector:95:info" width="4.948069219192242" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_ray:95:segment:95:intersections" width="1.6525744620819827" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_cross:95:join:95:tile:95:offsets" width="1.6472596125230254" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_add:95:missing:95:sectors" width="1.5444129873882029" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_sector:95:uv:95:summation" width="0.944231029241552" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_area:95:calc" width="0.8086817918769453" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_join:95:sector:95:angles" width="0.43530846695739717" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_point:95:update" width="0.3149116942386177" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_tile:95:sums" width="0.25322772170411945" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_all:95:sectors" width="0.18135638269241983" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_prepare:95:sector:95:angles" width="0.15263390699525486" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_xoyo:95:sector:95:creation" width="0.13360074493758897" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_normalize" width="0.09554813656305448" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_determine:95:tile" width="0.06076416066727872" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_prepare:95:df" width="0.006841582956457755" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_1_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">67.31</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">32</text><svg id="smallMultiples_45272_2" x="518" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_45272_2_encsvgid_s_sector:95:sums" width="166.7829117569088" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_hard:95:way:95:arctangents" width="143.24613305995084" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_separate:95:easy:95:hard:95:way" width="123.10929705797608" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_join:95:sector:95:info" width="15.910119595769206" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_cross:95:join:95:tile:95:offsets" width="5.222060000762088" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_ray:95:segment:95:intersections" width="1.6656815668814111" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_add:95:missing:95:sectors" width="1.5446881594379487" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_sector:95:uv:95:summation" width="0.8648117466223332" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_area:95:calc" width="0.7955292536861257" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_join:95:sector:95:angles" width="0.4249273656414338" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_point:95:update" width="0.2968634938169586" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_tile:95:sums" width="0.27007065140321507" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_all:95:sectors" width="0.17889697892070233" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_prepare:95:sector:95:angles" width="0.1507274190244289" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_xoyo:95:sector:95:creation" width="0.14078007801118045" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_normalize" width="0.09078963174018805" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_determine:95:tile" width="0.0631824172166043" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_prepare:95:df" width="0.006992456105228155" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_2_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">67.31</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">64</text><svg id="smallMultiples_45272_3" x="776" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_45272_3_encsvgid_s_sector:95:sums" width="142.71964749157573" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_separate:95:easy:95:hard:95:way" width="97.06328358851643" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_hard:95:way:95:arctangents" width="93.58572437359508" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_join:95:sector:95:info" width="48.98303796793705" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_cross:95:join:95:tile:95:offsets" width="15.427616121962068" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_ray:95:segment:95:intersections" width="1.6127208054940014" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_add:95:missing:95:sectors" width="1.5015384388896142" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_sector:95:uv:95:summation" width="0.8014664549837609" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_area:95:calc" width="0.7836051315304651" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_join:95:sector:95:angles" width="0.4060442194987385" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_tile:95:sums" width="0.30689055757359135" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_point:95:update" width="0.28943984911041504" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_all:95:sectors" width="0.1727266100295129" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_xoyo:95:sector:95:creation" width="0.15785274636863098" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_prepare:95:sector:95:angles" width="0.1541323516773609" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_normalize" width="0.090815348754183" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_determine:95:tile" width="0.06227632109018205" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_prepare:95:df" width="0.006711283418883318" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_3_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">67.31</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">128</text><svg id="smallMultiples_45272_4" x="1034" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_45272_4_encsvgid_s_sector:95:sums" width="174.1674554581406" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_join:95:sector:95:info" width="111.67935732941152" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_separate:95:easy:95:hard:95:way" width="92.3928758260249" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_hard:95:way:95:arctangents" width="58.14755564688048" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_cross:95:join:95:tile:95:offsets" width="33.90476776471905" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_ray:95:segment:95:intersections" width="1.5695582264386696" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_add:95:missing:95:sectors" width="1.4601846231519275" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_area:95:calc" width="0.7656898023477798" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_sector:95:uv:95:summation" width="0.7518231884017004" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_join:95:sector:95:angles" width="0.39961496599999985" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_tile:95:sums" width="0.31971134628387604" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_xoyo:95:sector:95:creation" width="0.29495357691093327" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_point:95:update" width="0.2831023196282584" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_all:95:sectors" width="0.1733729643145861" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_prepare:95:sector:95:angles" width="0.15368744733524817" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_normalize" width="0.096763694091216" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_determine:95:tile" width="0.06255749377652689" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_prepare:95:df" width="0.00857662416731736" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_45272_4_encsvgid_s_explode:95:points" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">explode_points</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">67.31</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">256</text><rect width="1291" height="337" x="0" y="0" fill="none" fill-opacity="0.0" stroke="#000000" /></svg>

# Updated Performance (Cross Products)
'<svg id="smallMultiples_1593" x="0" y="0" width="1292" height="338" xmlns="http://www.w3.org/2000/svg"><rect width="1291" height="337" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><svg id="smallMultiples_1593_0" x="2" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_1593_0_encsvgid_s_separate:95:easy:95:hard:95:way" width="242.0" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_sector:95:sums" width="156.6901546805121" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_hard:95:way:95:arctangents" width="110.19658950007548" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_medium:95:way:95:crossproducts" width="18.034961862651866" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_join:95:sector:95:info" width="1.3049817837494588" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_ray:95:segment:95:intersections" width="1.0828747888560204" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_add:95:missing:95:sectors" width="1.0022940355282235" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_sector:95:uv:95:summation" width="0.585400842572013" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_xoyo:95:sector:95:creation" width="0.5611407052883116" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_area:95:calc" width="0.5222375662635407" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_cross:95:join:95:tile:95:offsets" width="0.3509235186624059" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_join:95:sector:95:angles" width="0.29028325630801916" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_tile:95:sums" width="0.2240161967956531" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_point:95:update" width="0.20278815663679503" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_all:95:sectors" width="0.11736299249815689" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_prepare:95:sector:95:angles" width="0.10009392806436657" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_normalize" width="0.061074859208631004" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_determine:95:tile" width="0.04105260130130368" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_prepare:95:df" width="0.006045152634938635" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_0_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">medium_way_crossproducts</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">103.02</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><text x="244" text-anchor="end" y="316" font-family="Times" fill="#ff0000" font-size="12px" transform="rotate(90,244,316)">1 more</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">16</text><svg id="smallMultiples_1593_1" x="260" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_1593_1_encsvgid_s_separate:95:easy:95:hard:95:way" width="218.5869703918697" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_sector:95:sums" width="134.39961090029323" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_hard:95:way:95:arctangents" width="61.41949563414167" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_medium:95:way:95:crossproducts" width="29.165377092901817" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_join:95:sector:95:info" width="3.614298416090113" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_ray:95:segment:95:intersections" width="1.0501237715372713" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_cross:95:join:95:tile:95:offsets" width="1.0368316043483115" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_add:95:missing:95:sectors" width="0.97453584160585" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_sector:95:uv:95:summation" width="0.5317824556796292" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_area:95:calc" width="0.5029198481018825" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_xoyo:95:sector:95:creation" width="0.49395740807808014" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_join:95:sector:95:angles" width="0.26726922469531456" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_point:95:update" width="0.20013857194938584" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_tile:95:sums" width="0.1552446048964568" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_all:95:sectors" width="0.11639803066843675" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_prepare:95:sector:95:angles" width="0.09972429671926192" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_normalize" width="0.06099925279713232" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_determine:95:tile" width="0.0408241019243299" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_prepare:95:df" width="0.005690642572133729" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_1_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">medium_way_crossproducts</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">103.02</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><text x="244" text-anchor="end" y="316" font-family="Times" fill="#ff0000" font-size="12px" transform="rotate(90,244,316)">1 more</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">32</text><svg id="smallMultiples_1593_2" x="518" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_1593_2_encsvgid_s_separate:95:easy:95:hard:95:way" width="170.14590502148556" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_sector:95:sums" width="106.55731352586366" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_medium:95:way:95:crossproducts" width="30.052757783664443" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_hard:95:way:95:arctangents" width="25.072881005174906" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_join:95:sector:95:info" width="11.603501908469443" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_cross:95:join:95:tile:95:offsets" width="3.3506745405240195" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_ray:95:segment:95:intersections" width="1.0252055784497853" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_add:95:missing:95:sectors" width="0.9309725473952977" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_area:95:calc" width="0.5008583132816853" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_sector:95:uv:95:summation" width="0.48532931640734966" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_xoyo:95:sector:95:creation" width="0.481414584434196" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_join:95:sector:95:angles" width="0.25497562218562975" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_point:95:update" width="0.18838597531753598" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_tile:95:sums" width="0.1692866756779851" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_all:95:sectors" width="0.1152510534036272" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_prepare:95:sector:95:angles" width="0.09976237994875756" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_normalize" width="0.061183508422192064" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_determine:95:tile" width="0.03921340533566179" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_prepare:95:df" width="0.005851936249997573" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_2_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">medium_way_crossproducts</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">103.02</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><text x="244" text-anchor="end" y="316" font-family="Times" fill="#ff0000" font-size="12px" transform="rotate(90,244,316)">1 more</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">64</text><svg id="smallMultiples_1593_3" x="776" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_1593_3_encsvgid_s_separate:95:easy:95:hard:95:way" width="126.73050031210961" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_sector:95:sums" width="91.54049184449237" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_join:95:sector:95:info" width="35.312338521327106" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_medium:95:way:95:crossproducts" width="22.433899696942795" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_cross:95:join:95:tile:95:offsets" width="9.85559256402357" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_hard:95:way:95:arctangents" width="8.292549296404351" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_ray:95:segment:95:intersections" width="1.025714661620543" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_add:95:missing:95:sectors" width="0.9045545471226749" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_sector:95:uv:95:summation" width="0.4940184532547717" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_area:95:calc" width="0.4924133571410294" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_xoyo:95:sector:95:creation" width="0.4800889520192525" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_join:95:sector:95:angles" width="0.24667739848803052" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_tile:95:sums" width="0.1917860236450061" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_point:95:update" width="0.18282974414362213" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_all:95:sectors" width="0.11212878863247813" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_prepare:95:sector:95:angles" width="0.0983353789376566" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_normalize" width="0.06128767725581246" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_determine:95:tile" width="0.03958247663327385" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_prepare:95:df" width="0.005351253791628559" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_3_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">medium_way_crossproducts</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">103.02</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><text x="244" text-anchor="end" y="316" font-family="Times" fill="#ff0000" font-size="12px" transform="rotate(90,244,316)">1 more</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">128</text><svg id="smallMultiples_1593_4" x="1034" y="2" width="256" height="320" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="319" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><rect id="smallMultiples_1593_4_encsvgid_s_sector:95:sums" width="111.01835670674959" height="14" x="0" y="0" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_separate:95:easy:95:hard:95:way" width="109.33713109715056" height="14" x="0" y="15" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_join:95:sector:95:info" width="80.00596930554187" height="14" x="0" y="30" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_cross:95:join:95:tile:95:offsets" width="21.816501100929557" height="14" x="0" y="45" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_medium:95:way:95:crossproducts" width="13.406949482612275" height="14" x="0" y="60" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_hard:95:way:95:arctangents" width="2.5853713220521146" height="14" x="0" y="75" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_ray:95:segment:95:intersections" width="1.0323937880170855" height="14" x="0" y="90" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_add:95:missing:95:sectors" width="0.8870233804623558" height="14" x="0" y="105" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_xoyo:95:sector:95:creation" width="0.5606293819275835" height="14" x="0" y="120" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_area:95:calc" width="0.5107437115732641" height="14" x="0" y="135" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_sector:95:uv:95:summation" width="0.4945264163305442" height="14" x="0" y="150" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_join:95:sector:95:angles" width="0.24463602537756626" height="14" x="0" y="165" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_tile:95:sums" width="0.20766617029719112" height="14" x="0" y="180" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_point:95:update" width="0.1860068935690443" height="14" x="0" y="195" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_all:95:sectors" width="0.11272971959201947" height="14" x="0" y="210" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_prepare:95:sector:95:angles" width="0.10040027404280935" height="14" x="0" y="225" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_normalize" width="0.06314983516865023" height="14" x="0" y="240" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_determine:95:tile" width="0.039587517060707095" height="14" x="0" y="255" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_prepare:95:df" width="0.006107877954107908" height="14" x="0" y="270" fill="#4988b6" stroke="#4988b6"/><rect id="smallMultiples_1593_4_encsvgid_s_arctangents" width="0.0" height="14" x="0" y="285" fill="#4988b6" stroke="#4988b6"/><text x="2" text-anchor="start" y="13" font-family="Times" fill="#000000" font-size="12px">sector_sums</text><text x="2" text-anchor="start" y="28" font-family="Times" fill="#000000" font-size="12px">separate_easy_hard_way</text><text x="2" text-anchor="start" y="43" font-family="Times" fill="#000000" font-size="12px">join_sector_info</text><text x="2" text-anchor="start" y="58" font-family="Times" fill="#000000" font-size="12px">cross_join_tile_offsets</text><text x="2" text-anchor="start" y="73" font-family="Times" fill="#000000" font-size="12px">medium_way_crossproducts</text><text x="2" text-anchor="start" y="88" font-family="Times" fill="#000000" font-size="12px">hard_way_arctangents</text><text x="2" text-anchor="start" y="103" font-family="Times" fill="#000000" font-size="12px">ray_segment_intersections</text><text x="2" text-anchor="start" y="118" font-family="Times" fill="#000000" font-size="12px">add_missing_sectors</text><text x="2" text-anchor="start" y="133" font-family="Times" fill="#000000" font-size="12px">xoyo_sector_creation</text><text x="2" text-anchor="start" y="148" font-family="Times" fill="#000000" font-size="12px">area_calc</text><text x="2" text-anchor="start" y="163" font-family="Times" fill="#000000" font-size="12px">sector_uv_summation</text><text x="2" text-anchor="start" y="178" font-family="Times" fill="#000000" font-size="12px">join_sector_angles</text><text x="2" text-anchor="start" y="193" font-family="Times" fill="#000000" font-size="12px">tile_sums</text><text x="2" text-anchor="start" y="208" font-family="Times" fill="#000000" font-size="12px">point_update</text><text x="2" text-anchor="start" y="223" font-family="Times" fill="#000000" font-size="12px">all_sectors</text><text x="2" text-anchor="start" y="238" font-family="Times" fill="#000000" font-size="12px">prepare_sector_angles</text><text x="2" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">normalize</text><text x="2" text-anchor="start" y="268" font-family="Times" fill="#000000" font-size="12px">determine_tile</text><text x="2" text-anchor="start" y="283" font-family="Times" fill="#000000" font-size="12px">prepare_df</text><text x="2" text-anchor="start" y="298" font-family="Times" fill="#000000" font-size="12px">arctangents</text><text x="237" text-anchor="end" y="317" font-family="Times" fill="#000000" font-size="12px">103.02</text><text x="5" text-anchor="start" y="317" font-family="Times" fill="#000000" font-size="12px">subroutine_time</text><line x1="242" y1="2" x2="242" y2="320" stroke="#101010" stroke-width="1" stroke-dasharray="3 2" /><text x="244" text-anchor="middle" y="160.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(90,244,160.0)">subroutine</text><text x="244" text-anchor="end" y="316" font-family="Times" fill="#ff0000" font-size="12px" transform="rotate(90,244,316)">1 more</text><rect width="255" height="319" x="0" y="0" fill-opacity="0.0" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="334" font-family="Times" fill="#000000" font-size="14px">256</text><rect width="1291" height="337" x="0" y="0" fill="none" fill-opacity="0.0" stroke="#000000" /></svg>'